#### 基本语法
##### class
- 类，class 类名(需要继承的类)，类名中每个单词的首字母通常需要大写，默认继承object类，nn.Module是所有神经网络模型的基类
- __init__：
  - 类的构造函数，如果不写，会自动生成一个啥也不干的构造函数；
  - 这个函数在创建实例的时候会自动调用，用于做初始化的操作；
  - 它的第一个参数必须是self，表示当前实例的引用；
  - self后面为参数列表，任意数量，需要在创建实例的时候传入，如果定义了默认值，可以不传入；
- super([type[, object-or-type]])：
  - type: 子类的名称
  - object-or-type: 实例对象或类
  - 在大多数情况下，只需使用 super()，Python 会自动推断出正确的父类和实例

##### _set=set
- 这行代码将 Python 的内置 set 函数赋值给了一个名为 _set 的局部变量。这样做的主要目的是为了在代码中使用 _set 而不是直接使用 set，可能是为了在该函数中提高访问速度（尽管在现代 Python 中，这样的性能差异通常是微不足道的），或者是为了方便代码中的重命名

##### if not num_sample is None
- is的优先级高于not
- 如果num_sample不为空

##### random.sample
- 用于从一个序列（如列表、元组或字符串）中随机选择指定数量的元素，返回一个新列表（一个list类型）；无放回抽样
- arg1：采样的对象
- arg2：采样的数量

##### if的三元运算符
- <表达式1> if <条件> else <表达式2>
- _set(_sample(to_neigh, num_sample, )) if len(to_neigh) >= num_sample else to_neigh：如果集合中有多余num_sample个元素，则进行采样并去重；否则直接返回原始集合

##### 列表推导式
- samp_neighs = [ ... for to_neigh in to_neighs]：作用是遍历 to_neighs 列表中的每一个元素 to_neigh，并根据条件生成一个新的列表 samp_neighs

##### unique_nodes_list = list(set.union(*samp_neighs))
- *samp_neighs是 Python 的解包运算符，它会将 samp_neighs 列表中的每个 set 分别传递给 set.union() 方法作为独立的参数
- set.union()用于计算多个集合的并集。传递给它的所有集合中的元素会被合并到一个新的集合中

#####  mask = Variable(torch.zeros(len(samp_neighs), len(unique_nodes)))
- Variable 是 PyTorch 中一个曾经用于封装张量以支持自动求导的类。在旧版本的 PyTorch 中，所有需要计算梯度的张量都必须包装在 Variable 中
- 从 PyTorch 0.4 版本开始，Variable 和张量（Tensor）合并了，现在直接使用 Tensor 就可以支持自动求导功能

##### column_indices = [unique_nodes[n] for samp_neigh in samp_neighs for n in samp_neigh]   
- 最后生成一个列表[ s_n11, s_n12, .... ; s_n21, s_n22...]，第一个邻域的所有元素，第二个邻域的所有元素等

##### row_indices = [i for i in range(len(samp_neighs)) for j in range(len(samp_neighs[i]))]
- 生成对应数量的行下标

##### mask[row_indices, column_indices] = 1
- 高级索引（Advanced Indexing） 的特性，它允许你通过提供两个列表 row_indices 和 column_indices 来同时索引张量 mask 的多个位置，并将这些位置的值设置为 1
- 这两个列表的长度应该相同，每一对 (row_indices[i], column_indices[i]) 定位了 mask 中的一个具体元素

##### mask.cuda()
- cuda() 是 PyTorch 中的一个方法，用于将张量从 CPU 转移到 GPU 上。这个方法返回一个新的张量，存储在 GPU 的内存中
- 如果 mask 张量已经在 GPU 上，cuda() 不会再做任何改变；否则，它会将 mask 复制到 GPU

##### mask.sum(1, keepdim=True)
- 对张量中的元素进行求和，可以指定按照某个维度进行求和
- arg1：dim 要进行求和的维度，可选，未指定的话即对整个张量进行求和，返回一个标量
- arg2：keepdim 是否保持维度，默认为false，如果按照某一个维度求和之后，其实是可以减少一个维度的，如果这个为true，则不减少维度，将合并的那个维度大小设置为1
- arg3：dtype 和的类型，如果不指定，默认使用输入张量的 dtype
- mask.sum(1)，沿着第二个维度即列求和，相当于把列合并

##### mask = mask.div(num_neigh)
- 元素相除
- 需要形状相同的张量之间才可以进行逐元素的除法，这里先使用广播机制，将num_neigh广播为(m,n)的大小，再逐元素相除，这里就相当于做了平均的操作了，设置了权重

##### embed_matrix = self.features(torch.LongTensor(unique_nodes_list))
- torch.LongTensor(unique_nodes_list) 将 unique_nodes_list（这是一个包含唯一节点索引的列表）转换为一个 PyTorch 的长整型张量
- self.features = nn.Embedding(2708, 1433)
- self.features(torch.LongTensor(unique_nodes_list))它会返回一个形状为（len(unique_nodes_list), 1433)的张量，其中每一行对应于 unique_nodes_list 中每个节点的嵌入向量

##### to_feats = mask.mm(embed_matrix)
- 张量乘法

In [ ]:
class MeanAggregator(nn.Module):
    """
    Aggregates a node's embeddings using mean of neighbors' embeddings
    """
    def __init__(self, features, cuda=False, gcn=False): 
        """
        Initializes the aggregator for a specific graph.

        features -- function mapping LongTensor of node ids to FloatTensor of feature values.
        cuda -- whether to use GPU
        gcn --- whether to perform concatenation GraphSAGE-style, or add self-loops GCN-style
        """

        super(MeanAggregator, self).__init__()
        
        # 传入的是k-1层每个节点特征的嵌入表示
        self.features = features
        self.cuda = cuda
        self.gcn = gcn
        
    def forward(self, nodes, to_neighs, num_sample=10):
        """
        nodes --- list of nodes in a batch
        to_neighs --- list of sets, each set is the set of neighbors for node in batch
        num_sample --- number of neighbors to sample. No sampling if None.
        """
        # Local pointers to functions (speed hack)
        _set = set
        # 如果num_sample不为空，即需要对邻域进行采样
        if not num_sample is None:
            _sample = random.sample
            samp_neighs = [_set(_sample(to_neigh, 
                            num_sample,
                            )) if len(to_neigh) >= num_sample else to_neigh for to_neigh in to_neighs]
        else:
            # 不对邻域进行采样，直接使用所有邻居节点
            samp_neighs = to_neighs
        
        # 如果使用GCN的归纳式变体的话，聚合的时候不会做concat而是做union
        if self.gcn:
            samp_neighs = [samp_neigh | set([nodes[i]]) for i, samp_neigh in enumerate(samp_neighs)]
        unique_nodes_list = list(set.union(*samp_neighs))
        unique_nodes = {n:i for i,n in enumerate(unique_nodes_list)}
        mask = Variable(torch.zeros(len(samp_neighs), len(unique_nodes)))
        column_indices = [unique_nodes[n] for samp_neigh in samp_neighs for n in samp_neigh]   
        row_indices = [i for i in range(len(samp_neighs)) for j in range(len(samp_neighs[i]))]
        # 知道batch中每个节点的邻域是谁，即需要聚合的对象有哪些
        mask[row_indices, column_indices] = 1
        if self.cuda:
            mask = mask.cuda()
        num_neigh = mask.sum(1, keepdim=True)
        mask = mask.div(num_neigh)
        if self.cuda:
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list).cuda())
        else:
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list))
        # 生成聚合过后，第K层的h。在代码实现中，循环就是一个矩阵运算
        to_feats = mask.mm(embed_matrix)
        return to_feats
